In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import numpy  as np
import pandas as pd

import os

DETDIR='results/Main/'
ANNDIR='Annotations'
models= ['google','alex']
#models= ['resnet','alex']
#compid = {'google': 'comp4_52a76347-ada9-4968-9279-9b9635ed5553',
#          'alex'  : 'comp4_139dcae7-d1a3-4f30-b7e8-4c54028ae4a'}
compid = {'google': 'comp4_da952d79-5af5-44dc-83de-605c7c6d9794',
          'alex'  : 'comp4_139dcae7-d1a3-4f30-b7e8-4c54028ae4a'}
pars = ['','eminus','proton','pizero','muminus','gamma']

In [ ]:
detections = {}
for model in models:
    #detections = {'google' : [f for f in os.listdir(DETDIR) if f.startswith(compid['google'])],
    #              'alex'   : [f for f in os.listdir(DETDIR) if f.startswith(compid['alex'])] }
    detections[model] = [f for f in os.listdir(DETDIR) if f.startswith(compid[model])]


In [ ]:
pre_frames = {}
for model in models:
    frames = []
    for det in detections[model]:
        frames.append(pd.read_csv(os.path.join(DETDIR,det),
                                  delimiter=' ',names=['file','prob','xmin','ymin','xmax','ymax','pred_class']) )
    pre_frames[model] = frames

In [ ]:
df_frames = {}
for model in models:
    pred_df = pd.concat(pre_frames[model],ignore_index=True)
    df_frames[model] = pred_df

In [ ]:
#put the gt boxes into g
g = {}
for ix,image in enumerate(df_frames['google'].file.unique()):
    data = None
    with open(os.path.join(ANNDIR,image+".txt")) as f:
        data = f.read()
    data = data.split(" ")[2:]
    data = np.array(data,dtype=np.float64)
    g[image] = data

In [ ]:
#Check a single result
alex_df = df_frames['google']
alex_df[alex_df['file'] == 'eminus014928'].sort_values(by='prob',ascending=False).iloc[0:5]

In [ ]:
#Check a single result
alex_df = df_frames['alex']
alex_df[alex_df['file'] == 'eminus014928'].sort_values(by='prob',ascending=False).iloc[0:5]

In [ ]:
import re
truth = re.compile("^([a-z]+)[0-9]+")

def get_gt(row): 
    return pars.index(truth.search(row.file).group(1))

In [ ]:
def iou(bbox1,bbox2):
    ixmin = np.maximum(bbox1[0], bbox2[0])
    iymin = np.maximum(bbox1[1], bbox2[1])
    ixmax = np.minimum(bbox1[2], bbox2[2])
    iymax = np.minimum(bbox1[3], bbox2[3])

    iw = np.maximum(ixmax - ixmin, 0.)
    ih = np.maximum(iymax - iymin, 0.)

    inters = iw*ih
             # union                                                                                                        
    uni = ((bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1]) +
           (bbox2[2] - bbox2[0]) *
           (bbox2[3] - bbox2[1]) - inters)
             # IoU                                                                                                          
    iou = inters / uni
    return iou

def iou_row(row):
    pred_bbox = np.array(row[['xmin','ymin','xmax','ymax']])
    return iou(pred_bbox,g[row['file']])


In [ ]:
# Go row by row and mark the groundtruth

for model in models:
    the_df = df_frames[model]
    the_df['gt'] = the_df.apply(get_gt,axis=1)
# Go row by row and calculate iou, very slowly
    the_df['iou'] = the_df.apply(iou_row,axis=1)

In [ ]:
ious = []
probs = []
im_df = {}
for model in models:
    pred_df = df_frames[model]
    im = {}
    for ix,image in enumerate(pred_df.file.unique()):
        df = pred_df[pred_df['file'] == image].sort_values(by='prob',ascending=False)
        top = df.iloc[0] # take only the top prediction
        im[image] = {'gt':top['gt'],'top_pred_class':top['pred_class'],'top_iou':top['iou'],'top_prob':top['prob']}
        
    im_df[model] = pd.DataFrame(im).T

In [ ]:
ar = lambda x, y : np.arange(0,x,y)

In [ ]:
matplotlib.rcParams['font.size']=14
matplotlib.rcParams['font.family']='serif'

In [ ]:
inc = 0.01

npars = len(pars) - 1
for pred in xrange(npars):
    pred += 1
    a = {}
    c = 0
    fig,ax = plt.subplots(figsize=(15,6))
    for model in models:
        c+=1
        model_df = im_df[model]
        ax = plt.subplot(1,2,c)

        for i in xrange(npars):
            i += 1    
            res = []
            for j in ar(1,inc):
                bot = model_df[model_df['top_pred_class'] == pred].query('top_prob > {}'.format(j))
                top = bot.query('gt == {}'.format(i))
                bot = float(bot.values.size)
                top = float(top.values.size)
                r = top/bot if bot > 0.0 else 0.0
                res.append(r * 100.0)
                a[i] = res


        ax.hist([ar(1,inc) for _ in range(npars)],bins=ar(1+inc,inc),
                weights=[a[r] for r in a],
                stacked=True,histtype='stepfilled',alpha=0.65,label=[pars[r] for r in a])
        ax.set_ylim(0,100)
        ax.legend(loc='best',fontsize=12)
        
        ax.set_title("Detection: {} - Model: {}".format(pars[pred],model),fontweight='bold')
        ax.set_ylabel("Percent Truth",fontweight='bold')
        ax.set_xlabel("Prob. Threshold",fontweight='bold')
    plt.savefig('{}_google5_4b_alt_opt_detection_truth_{}_{}.pdf'.format(model,pars[pred],npars), format='pdf', dpi=1000)
    plt.show()

In [ ]:
inc = 0.01

npars = len(pars) - 1
for pred in xrange(npars):
    pred += 1
    a = {}
    c = 0
    fig,ax = plt.subplots(figsize=(15,6))
    for model in models:
        c+=1
        model_df = im_df[model]
        ax = plt.subplot(1,2,c)

        for i in xrange(npars):
            i += 1    
            res = []
            for j in ar(1,inc):
                bot = model_df[model_df['gt'] == pred].query('top_prob > {}'.format(j))
                top = bot.query('top_pred_class == {}'.format(i))
                bot = float(bot.values.size)
                top = float(top.values.size)
                r = top/bot if bot > 0.0 else 0.0
                res.append(r * 100.0)
                a[i] = res

        ax.hist([ar(1,inc) for _ in range(npars)],bins=ar(1+inc,inc),
                weights=[a[r] for r in a],
                stacked=True,histtype='stepfilled',alpha=0.65,label=[pars[r] for r in a])
        ax.set_ylim(0,100)
        ax.legend(loc='best',fontsize=12)
        
        ax.set_title("Truth: {} - Model: {}".format(pars[pred],model),fontweight='bold')
        ax.set_ylabel("Percent Detected",fontweight='bold')
        ax.set_xlabel("Prob. Threshold",fontweight='bold')
    plt.savefig('{}_google__truth_alt_opt_detection_{}_{}.pdf'.format(model,pars[pred],npars), format='pdf', dpi=1000)
    plt.show()

In [ ]:
tparts = {}
for model in models:
    the_df = im_df[model]
    true_particle = [the_df[the_df['gt']==i] for i in xrange(len(pars)) if i != 0]
    tparts[model] = true_particle

pthresh = 0.50
c = 0
for model in models:
    #fig, ax = plt.subplots(figsize=(10,6))
    c +=1
    #ax = plt.subplot(1,2,c)
    for ix, tpar in enumerate(tparts[model]):
        fig, ax = plt.subplots(figsize=(10,6))
        ii  = ix
        ii += 1
        divs = 0.01
        ntot = float(tpar.index.values.size)
        for i in xrange(5):
            i+=1
            s = tpar[ tpar['top_pred_class'] == i ][ tpar['top_prob'] >= pthresh]
            nn = s.top_iou.values.size
            ax.hist(s.top_iou.values,
                    bins=np.arange(0,1.0+divs,divs),
                    label='Detected: {}\nFrac: {:.2f}'.format(pars[i],
                                                          nn/ntot),
                    alpha=0.5)
        
        ax.set_title("Truth: {} Model: {}".format(pars[ii],model),fontweight='bold')
        ax.set_xlabel("IoU @ Prob Thresh: {}".format(pthresh),fontweight='bold')
        ax.legend(loc='best',fontsize=14)
        plt.tight_layout()
        plt.savefig('IOU_{}_google_updated_{}.pdf'.format(model,pars[ii]), format='pdf', dpi=1000) 
        plt.show()